In [3]:
import yfinance as yf
import pandas as pd
from ta.momentum import RSIIndicator

def trading_strategy(symbol):
    try:
        # Retrieve real-time stock data and option data
        ticker = yf.Ticker(symbol)
        stock_data = ticker.history(period='1w')
        option_chain = ticker.option_chain()

        # Calculate RSI for the stock price
        rsi_period = 14  # RSI period
        stock_data['RSI'] = RSIIndicator(stock_data['Close'], window=rsi_period).rsi()

        # Extract implied volatility of call options
        call_options = option_chain.calls
        call_options['Implied Volatility'] = call_options['impliedVolatility']

        # Define trading signals
        buy_signal = False
        sell_signal = False

        # Define trading strategy based on RSI and implied volatility
        if stock_data['RSI'].iloc[-1] < 30:  # If RSI is below 30 (oversold)
            if call_options['Implied Volatility'].mean() > 0.2:  # If implied volatility of call options is high
                buy_signal = True
        elif stock_data['RSI'].iloc[-1] > 70:  # If RSI is above 70 (overbought)
            if call_options['Implied Volatility'].mean() < 0.1:  # If implied volatility of call options is low
                sell_signal = True

        # Execute trades based on signals
        if buy_signal:
            print(f"Buy signal triggered for {symbol}.")
            # Execute buy orders for call options or other appropriate trading actions
        elif sell_signal:
            print(f"Sell signal triggered for {symbol}.")
            # Execute sell orders for call options or other appropriate trading actions
        else:
            print(f"No trade signal detected for {symbol}.")
    except Exception as e:
        print(f"An error occurred for symbol {symbol}: {e}")

# List of symbols
symbols = ['AAPL', 'AMZN', 'GOOGL']

# Apply trading strategy for each symbol
for symbol in symbols:
    trading_strategy(symbol)

No trade signal detected for AAPL.
No trade signal detected for AMZN.
No trade signal detected for GOOGL.


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np

# Function to implement moving average crossover strategy
def moving_average_crossover_strategy(data, short_window, long_window):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0
    
    # Create short simple moving average
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()

    # Create long simple moving average
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()
    
    # Generate signals
    signals['signal'][short_window:] = np.where(
        signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)
    
    # Generate trading orders
    signals['positions'] = signals['signal'].diff()
    return signals

# Define risk management parameters
risk_per_trade = 0.02  # 2% risk per trade
initial_portfolio_value = 100000  # $100,000 initial portfolio value

# Define position sizing function
def position_size(portfolio_value, risk_per_trade, stop_loss):
    return (portfolio_value * risk_per_trade) / stop_loss

# Define backtesting function
def backtest_strategy(signals, data, initial_portfolio_value, risk_per_trade):
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['positions'] = signals['positions']
    portfolio['close'] = data['Close']
    portfolio['holdings'] = signals['positions'].cumsum()
    portfolio['cash'] = initial_portfolio_value - (portfolio['positions'] * portfolio['close']).cumsum()

    # Calculate total portfolio value
    portfolio['total'] = portfolio['cash'] + (portfolio['positions'] * portfolio['close'])

    # Calculate returns
    portfolio['returns'] = portfolio['total'].pct_change()

    # Calculate risk-adjusted returns
    portfolio['risk_adjusted_returns'] = portfolio['returns'] / risk_per_trade

    return portfolio

# Define error handling
def get_option_chain(ticker, expiration_date):
    try:
        options_chain = ticker.option_chain(expiration_date)
        return options_chain.calls, options_chain.puts
    except Exception as e:
        print(f"Error retrieving option chain for {ticker.symbol} with expiration date {expiration_date}: {e}")
        return None, None

# Define the stock symbols
symbols = ['AAPL', 'AMZN']

# Retrieve real-time option data and advanced option statistics
option_data = {}
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    expirations = ticker.options

    option_data[symbol] = {}
    for expiration_date in expirations:
        calls, puts = get_option_chain(ticker, expiration_date)
        if calls is not None and puts is not None:
            option_data[symbol][expiration_date] = {
                'calls': calls,
                'puts': puts
            }

            # Implement moving average crossover strategy for backtesting
            historical_data = ticker.history(period="1y")
            signals = moving_average_crossover_strategy(historical_data, short_window=50, long_window=200)
            portfolio = backtest_strategy(signals, historical_data, initial_portfolio_value, risk_per_trade)

            # Print portfolio statistics
            print(f"Symbol: {symbol}, Expiration Date: {expiration_date}")
            print(portfolio.tail())  # Print last few rows of the portfolio for analysis


Symbol: AAPL, Expiration Date: 2024-03-08
                           positions       close  holdings         cash  \
Date                                                                      
2024-02-26 00:00:00-05:00        0.0  181.160004       1.0  99828.10881   
2024-02-27 00:00:00-05:00        0.0  182.630005       1.0  99828.10881   
2024-02-28 00:00:00-05:00        0.0  181.419998       1.0  99828.10881   
2024-02-29 00:00:00-05:00        0.0  180.750000       1.0  99828.10881   
2024-03-01 00:00:00-05:00        0.0  179.660004       1.0  99828.10881   

                                 total  returns  risk_adjusted_returns  
Date                                                                    
2024-02-26 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-27 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-28 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-29 00:00:00-05:00  99828.10881      0.0                    

Symbol: AAPL, Expiration Date: 2024-08-16
                           positions       close  holdings         cash  \
Date                                                                      
2024-02-26 00:00:00-05:00        0.0  181.160004       1.0  99828.10881   
2024-02-27 00:00:00-05:00        0.0  182.630005       1.0  99828.10881   
2024-02-28 00:00:00-05:00        0.0  181.419998       1.0  99828.10881   
2024-02-29 00:00:00-05:00        0.0  180.750000       1.0  99828.10881   
2024-03-01 00:00:00-05:00        0.0  179.660004       1.0  99828.10881   

                                 total  returns  risk_adjusted_returns  
Date                                                                    
2024-02-26 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-27 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-28 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-29 00:00:00-05:00  99828.10881      0.0                    

Symbol: AAPL, Expiration Date: 2025-09-19
                           positions       close  holdings         cash  \
Date                                                                      
2024-02-26 00:00:00-05:00        0.0  181.160004       1.0  99828.10881   
2024-02-27 00:00:00-05:00        0.0  182.630005       1.0  99828.10881   
2024-02-28 00:00:00-05:00        0.0  181.419998       1.0  99828.10881   
2024-02-29 00:00:00-05:00        0.0  180.750000       1.0  99828.10881   
2024-03-01 00:00:00-05:00        0.0  179.660004       1.0  99828.10881   

                                 total  returns  risk_adjusted_returns  
Date                                                                    
2024-02-26 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-27 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-28 00:00:00-05:00  99828.10881      0.0                    0.0  
2024-02-29 00:00:00-05:00  99828.10881      0.0                    

Symbol: AMZN, Expiration Date: 2024-04-05
                           positions       close  holdings          cash  \
Date                                                                       
2024-02-26 00:00:00-05:00        0.0  174.729996       1.0  99889.739998   
2024-02-27 00:00:00-05:00        0.0  173.539993       1.0  99889.739998   
2024-02-28 00:00:00-05:00        0.0  173.160004       1.0  99889.739998   
2024-02-29 00:00:00-05:00        0.0  176.759995       1.0  99889.739998   
2024-03-01 00:00:00-05:00        0.0  178.220001       1.0  99889.739998   

                                  total  returns  risk_adjusted_returns  
Date                                                                     
2024-02-26 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-27 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-28 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-29 00:00:00-05:00  99889.739998      0.0       

Symbol: AMZN, Expiration Date: 2024-12-20
                           positions       close  holdings          cash  \
Date                                                                       
2024-02-26 00:00:00-05:00        0.0  174.729996       1.0  99889.739998   
2024-02-27 00:00:00-05:00        0.0  173.539993       1.0  99889.739998   
2024-02-28 00:00:00-05:00        0.0  173.160004       1.0  99889.739998   
2024-02-29 00:00:00-05:00        0.0  176.759995       1.0  99889.739998   
2024-03-01 00:00:00-05:00        0.0  178.220001       1.0  99889.739998   

                                  total  returns  risk_adjusted_returns  
Date                                                                     
2024-02-26 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-27 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-28 00:00:00-05:00  99889.739998      0.0                    0.0  
2024-02-29 00:00:00-05:00  99889.739998      0.0       